# PREPROCESSING 




In [3]:
import pandas as pd
import numpy as np
from sklearn import neighbors as nei
from sklearn import metrics as met

from sklearn import decomposition as dec
from sklearn import manifold as man

In [4]:
data = pd.read_csv("train.csv")
data.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [5]:
Z = data.pivot("movieID", "userID", "rating")
Z_norm = Z - Z.mean(axis=1).values.reshape(-1,1)
Z_norm.fillna(0, inplace=True)
Z_norm.head()


userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.0,0.0,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.0,0.0,0.854713,-1.145287,0.0,0.0,0.0,0.0,-0.145287,0.854713,0.0,0.854713,0.0,0.0,0.854713,0.0,0.0,0.0,-2.145287,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.854713,0.0,0.854713,0.0,0.854713,0.0,0.0,0.0,0.0,0.0,-1.145287,0.0,-1.145287,0.0,0.0,-0.145287,0.0,-1.145287,0.0,0.854713,-0.145287,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.854713,0.854713,-0.145287,0.0,-0.145287,0.0,0.0,0.0,0.0,0.854713
2,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.8,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,-1.200000,0.0,0.0,0.0,0.0,-0.2,...,1.8,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,-2.2,0.000000,0.0,0.0,-1.200000,0.0,0.000000,0.0,0.000000,-1.200000,0.0,0.0,0.0,0.0,-0.2,0.0,0.0,1.800000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.0,0.0,0.0,0.0,-2.021142,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,-0.021142,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-1.021142,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-1.021142,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,-0.733728,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.266272,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.266272,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.266272,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.0,0.0,0.0,0.0,-2.017007,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.982993,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


In [11]:
U = data.pivot("userID", "movieID", "rating")
U.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train_size = 3670
train = Z_norm[:train_size]
test = Z_norm[train_size:]

In [45]:
train_mean

userID
1      -0.027327
2      -0.002775
3      -0.000987
4      -0.014665
5      -0.067375
          ...   
6036   -0.029662
6037    0.000813
6038    0.001627
6039   -0.001685
6040    0.003811
Length: 6040, dtype: float64

In [8]:
train_mean = train.mean(axis=0)
centered_train = train - train_mean
centered_train.shape

(3670, 6040)

In [9]:
U, S, VT = np.linalg.svd(centered_train.T, full_matrices=False)

In [10]:
np.set_printoptions(suppress=True)

squares = np.array(list(map(lambda x:x**2, S)))
total = squares.sum()

for i in range(1, len(squares)):
  r = squares[:i].sum() / total
  print(i, r)

1 0.024999585649799545
2 0.04007572805056297
3 0.04896307789681224
4 0.05660261266982928
5 0.06384195123802072
6 0.07042903862798068
7 0.07654589958006731
8 0.08205747646525635
9 0.08684945149710742
10 0.09148293838765666
11 0.0956939113766779
12 0.0995977831806189
13 0.10333964502203212
14 0.10704617899537196
15 0.11061702933079412
16 0.11413486655479896
17 0.11755411170932252
18 0.1208644098672993
19 0.12404306419672116
20 0.1271811399359001
21 0.13028540993768847
22 0.1333165489954017
23 0.1362486712516633
24 0.13916725174411398
25 0.14205417821641014
26 0.14489247670134137
27 0.1477008074516336
28 0.1504607324843725
29 0.15317793839978372
30 0.1558603088333641
31 0.1585320838840251
32 0.16115449164721896
33 0.16373336995616777
34 0.1663099748542078
35 0.16887200391242976
36 0.17141276340840647
37 0.17392528433993995
38 0.17641689130063062
39 0.17888577819648707
40 0.1813332720441416
41 0.1837644868741346
42 0.18616748584372633
43 0.18854469865993295
44 0.19090474035125335
45 0.1932

**Between 700 and 1100 seems to usable for %80 and %90 percent of sum of squares**

**I did create a data for lsa approach and I also did create a small test to see if my model is working and my rmse results for this pairs(not best test approach but still usable for me to see if there is any problem)**

#LSA METHOD 

In [26]:
a=[700,800,900,1000,1100]
for i in a:
 svd = dec.TruncatedSVD(n_components=i)
 temp = svd.fit_transform(centered_train)
 myPreds = []
 realLabels = []
 for movieID in test.index:
  for userID in test.columns:
      if not np.isnan(Z.loc[movieID, userID]):
        new_user = test.loc[movieID:movieID, :]
        new_user_centered = new_user - train_mean
        new_user_red = svd.transform(new_user_centered)
        y=list(test.columns).index(userID)
        X=new_user_red.dot(svd.components_.T[y])
        y=Z.loc[movieID:movieID, :]
        y=y.mean(axis=1).values.reshape(-1,1)
        pred=y+X
        if pred>5:
         pred=5
        if pred<1:
         pred=1   
        myPreds.append(pred)
        real_label = Z.loc[movieID, userID]
        realLabels.append(real_label)  
 print(i,met.mean_squared_error(realLabels, myPreds, squared=False))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


700 0.7618370091479156


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


800 0.7394673424919864


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


900 0.7188907633339988


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


1000 0.6984530921366594
1100 0.6793095093772372


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


**Results seem very good but I will try 1100.Better results than other approaches.**

# CREATING KAGGLE SUBMISSION

In [51]:
test = pd.read_csv("test_without_labels.csv")

In [28]:
 svd = dec.TruncatedSVD(n_components=1100)
 temp = svd.fit_transform(centered_train)

In [32]:

myPreds = []
for i in test.IDs:
  a = i.split("_")
  userID=int(a[0])
  movieID=int(a[1])
  new_user = Z_norm.loc[movieID:movieID, :]
  new_user_centered = new_user - train_mean
  new_user_red = svd.transform(new_user_centered)
  y=list(train.columns).index(userID)
  X=new_user_red.dot(svd.components_.T[y])
  y=Z.loc[movieID:movieID, :]
  y=y.mean(axis=1).values.reshape(-1,1)
  pred=y+X
  if pred>5:
    pred=5
  if pred<1:
    pred=1   
  myPreds.append(pred)

In [33]:
myPreds_num=[]

In [34]:
for i in myPreds:
  i=float(i)
  myPreds_num.append(i)

In [35]:
test["rating"]=myPreds_num

In [36]:
test

,IDs,rating
0,629_2683,3.373313
1,601_904,4.496698
2,5673_3717,2.947612
3,5616_1721,3.595530
4,1099_3697,2.910024
...,...,...
9994,4198_3671,3.955081
9995,566_3148,3.941982
9996,5531_2105,3.387852
9997,4650_377,3.551775


In [38]:

dfpred = pd.DataFrame(test, columns = ['IDs', "rating"])
dfpred.to_csv("output.xlsx",index=False)

**Result is 0.98 for caggle worse of all approaches I will try now for n compenents 800.**

In [58]:
 svd = dec.TruncatedSVD(n_components=800)
 temp = svd.fit_transform(centered_train)

In [59]:

myPreds = []
for i in test.IDs:
  a = i.split("_")
  userID=int(a[0])
  movieID=int(a[1])
  new_user = Z_norm.loc[movieID:movieID, :]
  new_user_centered = new_user - train_mean
  new_user_red = svd.transform(new_user_centered)
  y=list(train.columns).index(userID)
  X=new_user_red.dot(svd.components_.T[y])
  y=Z.loc[movieID:movieID, :]
  y=y.mean(axis=1).values.reshape(-1,1)
  pred=y+X
  if pred>5:
    pred=5
  if pred<1:
    pred=1   
  myPreds.append(pred)

In [60]:
myPreds_num=[]

In [61]:
for i in myPreds:
  i=float(i)
  myPreds_num.append(i)
 

In [62]:
test["rating"]=myPreds_num

In [63]:

dfpred = pd.DataFrame(test, columns = ['IDs', "rating"])
dfpred.to_csv("output.xlsx",index=False)

**Result is again 0.98 not good**